# Why use BLP and Logit Tutorial

This code is taken directly from the pyBLP logit tutorial with minimal modifications: https://pyblp.readthedocs.io/en/stable/_notebooks/tutorial/logit_nested.html

In [1]:
import pyblp
import numpy as np
import pandas as pd
#from sklearn.linear_model import LinearRegression
import statsmodels as sm
import statsmodels.regression.linear_model as smrl

pyblp.options.digits = 2
pyblp.options.verbose = False
pyblp.__version__

'1.1.0'

In this tutorial, we'll use data from [BLP (1995)]

We will first replicate the results of the linear regression in BLP Table 3, Column 1.  Then we run a plain (IIA) logit model on the automobile dataset.

## Theory of Plain Logit

Let's start with the plain logit model under independence of irrelevant alternatives (IIA). In this  model (indirect) utility is given by

$$U_{ijt} = \alpha p_{jt} + x_{jt} \beta^\text{ex} + \xi_{jt} + \epsilon_{ijt},$$

where $\epsilon_{ijt}$ is distributed IID with the Type I Extreme Value (Gumbel) distribution. It is common to normalize the mean utility of the outside good to zero so that $U_{i0t} = \epsilon_{i0t}$. This gives us aggregate market shares

$$s_{jt} = \frac{\exp(\alpha p_{jt} + x_{jt} \beta^\text{ex} + \xi_{jt})}{1 + \sum_k \exp(\alpha p_{kt} + x_{kt} \beta^\text{ex} + \xi_{kt})}.$$

If we take logs we get

$$\log s_{jt} = \alpha p_{jt} + x_{jt} \beta^\text{ex} + \xi_{jt} - \log \sum_k \exp(\alpha p_{kt} + x_{kt} \beta^\text{ex} + \xi_{kt})$$

and

$$\log s_{0t} = -\log \sum_k \exp(\alpha p_{kt} + x_{kt} \beta^\text{ex} + \xi_{kt}).$$

By differencing the above we get a linear estimating equation:

$$\log s_{jt} - \log s_{0t} = \alpha p_{jt} + x_{jt} \beta^\text{ex} + \xi_{jt}.$$

Because the left hand side is data, we can estimate this model using linear IV GMM.

## Linear Regresion Limitations

In [2]:
# read in the product data
product_data = pd.read_csv(pyblp.data.BLP_PRODUCTS_LOCATION)

# Calculate the share_out variable
product_data['share_out'] = 1.0 - product_data.groupby('market_ids')['shares'].transform('sum')

# Calculate the dif_2 variable as in the original paper
# dif_2 represents the logarithmic difference between the market share of a specific product and the share of 
# the outside option in a given market.
product_data['dif_2'] = np.log(product_data['shares']) - np.log(product_data['share_out'])

# Explore the dataset
product_data.head()

,market_ids,clustering_ids,car_ids,firm_ids,region,shares,prices,hpwt,air,mpd,...,supply_instruments4,supply_instruments5,supply_instruments6,supply_instruments7,supply_instruments8,supply_instruments9,supply_instruments10,supply_instruments11,share_out,dif_2
0,1971,AMGREM71,129,15,US,0.001051,4.935802,0.528997,0,1.888146,...,1.595656,87.0,-61.959985,0.0,46.060389,29.786989,0.0,1.888146,0.880106,-6.730022
1,1971,AMHORN71,130,15,US,0.000670,5.516049,0.494324,0,1.935989,...,1.490295,87.0,-61.959985,0.0,46.060389,29.786989,0.0,1.935989,0.880106,-7.180407
2,1971,AMJAVL71,132,15,US,0.000341,7.108642,0.467613,0,1.716799,...,1.357703,87.0,-61.959985,0.0,46.060389,29.786989,0.0,1.716799,0.880106,-7.857303
3,1971,AMMATA71,134,15,US,0.000522,6.839506,0.426540,0,1.687871,...,1.261347,87.0,-61.959985,0.0,46.060389,29.786989,0.0,1.687871,0.880106,-7.429667
4,1971,AMAMBS71,136,15,US,0.000442,8.928395,0.452489,0,1.504286,...,1.237365,87.0,-61.959985,0.0,46.060389,29.786989,0.0,1.504286,0.880106,-7.595531


In [3]:
# Define predictor variables (X) and response variable (y)
X = product_data[["hpwt", "air", "mpd", "space", "prices"]]
y = product_data["dif_2"]

# Adding a constant term to the predictor variables (intercept)
X = X.assign(intercept=1)

# Create a linear regression model
model = smrl.OLS(y, X)

# Fit the model
results = model.fit()

# Print the regression summary
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  dif_2   R-squared:                       0.387
Model:                            OLS   Adj. R-squared:                  0.386
Method:                 Least Squares   F-statistic:                     279.2
Date:                Wed, 16 Aug 2023   Prob (F-statistic):          6.52e-232
Time:                        09:06:57   Log-Likelihood:                -3319.4
No. Observations:                2217   AIC:                             6651.
Df Residuals:                    2211   BIC:                             6685.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
hpwt          -0.1243      0.277     -0.448      0.6

In [4]:
# The same magnitude price increase for a Yugo and BMW decrease demand equivalently
price_change = 0.02  # Change in car price
price_change_coefficient = results.params['prices']  # Coefficient for prices variable

# Assuming a Yugo and a BMW both have the same values for other predictors,
# their changes in the dependent variable (dif_2) would be the same.
yugo_change = price_change * price_change_coefficient
bmw_change = price_change * price_change_coefficient

print("Change in dif_2 for Yugo:", yugo_change)
print("Change in dif_2 for BMW:", bmw_change)


Change in dif_2 for Yugo: -0.001772785166011831
Change in dif_2 for BMW: -0.001772785166011831


## Application of Plain Logit

A Logit [`Problem`](https://pyblp.readthedocs.io/en/stable/_api/pyblp.Problem.html#pyblp.Problem) can be created by simply excluding the formulation for the nonlinear parameters, $X_2$, along with any agent information. In other words, it requires only specifying the _linear component_ of demand.

We'll set up and solve a simple logit for the automobile data. Since we won't include any demand-side nonlinear characteristics or parameters, we don't have to worry about configuring an optimization routine.

There are some important reserved variable names:

- `market_ids` are the unique market identifiers which we subscript with $t$.
- `shares` specifies the market shares which need to be between zero and one, and within a market ID, $\sum_{j} s_{jt} \leq 1$.
- `prices` are prices $p_{jt}$. These have some special properties and are _always_ treated as endogenous.
- `demand_instruments0`, `demand_instruments1`, and so on are numbered demand instruments. These represent only the _excluded_ instruments. The exogenous regressors in $X_1$ will be automatically added to the set of instruments.

We begin with two steps:

1. Load the product data which at a minimum consists of `market_ids`, `shares`, `prices`, and at least a single column of demand instruments, `demand_instruments0`.
2. Define a [`Formulation`](https://pyblp.readthedocs.io/en/stable/_api/pyblp.Formulation.html#pyblp.Formulation) for the $X_1$ (linear) demand model.

    - This and all other formulas are similar to R and [patsy](https://patsy.readthedocs.io/en/stable/) formulas.
    - It includes a constant by default. To exclude the constant, specify either a `0` or a `-1`.
    - To efficiently include fixed effects, use the `absorb` option and specify which categorical variables you would like to absorb.
    - Some model reduction may happen automatically. The constant will be excluded if you include fixed effects and some precautions are taken against collinearity. However, you will have to make sure that differently-named variables are not collinear.
    
3. Combine the [`Formulation`](https://pyblp.readthedocs.io/en/stable/_api/pyblp.Formulation.html#pyblp.Formulation) and product data to construct a [`Problem`](https://pyblp.readthedocs.io/en/stable/_api/pyblp.Problem.html#pyblp.Problem).
4. Use [`Problem.solve`](https://pyblp.readthedocs.io/en/stable/_api/pyblp.Problem.solve.html#pyblp.Problem.solve) to estimate paramters.

### Loading the Data

The `product_data` argument of [`Problem`](https://pyblp.readthedocs.io/en/stable/_api/pyblp.Problem.html#pyblp.Problem) should be a structured array-like object with fields that store data. Product data can be a structured [NumPy](https://numpy.org/) array, a [pandas](https://pandas.pydata.org/) DataFrame, or other similar objects.

In [5]:
#product_data = pd.read_csv(pyblp.data.NEVO_PRODUCTS_LOCATION)
#product_data.head()

product_data = pd.read_csv(pyblp.data.BLP_PRODUCTS_LOCATION)
product_data.head()

,market_ids,clustering_ids,car_ids,firm_ids,region,shares,prices,hpwt,air,mpd,...,supply_instruments2,supply_instruments3,supply_instruments4,supply_instruments5,supply_instruments6,supply_instruments7,supply_instruments8,supply_instruments9,supply_instruments10,supply_instruments11
0,1971,AMGREM71,129,15,US,0.001051,4.935802,0.528997,0,1.888146,...,0.0,1.705933,1.595656,87.0,-61.959985,0.0,46.060389,29.786989,0.0,1.888146
1,1971,AMHORN71,130,15,US,0.000670,5.516049,0.494324,0,1.935989,...,0.0,1.680910,1.490295,87.0,-61.959985,0.0,46.060389,29.786989,0.0,1.935989
2,1971,AMJAVL71,132,15,US,0.000341,7.108642,0.467613,0,1.716799,...,0.0,1.801067,1.357703,87.0,-61.959985,0.0,46.060389,29.786989,0.0,1.716799
3,1971,AMMATA71,134,15,US,0.000522,6.839506,0.426540,0,1.687871,...,0.0,1.818061,1.261347,87.0,-61.959985,0.0,46.060389,29.786989,0.0,1.687871
4,1971,AMAMBS71,136,15,US,0.000442,8.928395,0.452489,0,1.504286,...,0.0,1.933210,1.237365,87.0,-61.959985,0.0,46.060389,29.786989,0.0,1.504286


The product data contains `market_ids`, `product_ids`, `firm_ids`, `shares`, `prices`, a number of other IDs and product characteristics, and some pre-computed excluded `demand_instruments0`, `demand_instruments1`, and so on. The `product_ids` will be incorporated as fixed effects. 

For more information about the instruments and the example data as a whole, refer to the [`data`](https://pyblp.readthedocs.io/en/stable/_api/pyblp.data.html#module-pyblp.data) module.

### Setting Up the Problem

We can combine the [`Formulation`](https://pyblp.readthedocs.io/en/stable/_api/pyblp.Formulation.html#pyblp.Formulation) and `product_data` to construct a [`Problem`](https://pyblp.readthedocs.io/en/stable/_api/pyblp.Problem.html#pyblp.Problem). We pass the [`Formulation`](https://pyblp.readthedocs.io/en/stable/_api/pyblp.Formulation.html#pyblp.Formulation) first and the `product_data` second. We can also display the properties of the problem by typing its name. 

In [7]:
#logit_formulation = pyblp.Formulation('prices', absorb='C(product_ids)')
#logit_formulation


logit_formulation = pyblp.Formulation('prices + hpwt + air + mpd + space', absorb=None)
logit_formulation

1 + prices + hpwt + air + mpd + space

In [8]:
problem = pyblp.Problem(logit_formulation, product_data)
problem

Dimensions:
 T    N     F    K1    MD 
---  ----  ---  ----  ----
20   2217  26    6     13 

Formulations:
     Column Indices:         0     1      2     3    4     5  
--------------------------  ---  ------  ----  ---  ---  -----
X1: Linear Characteristics   1   prices  hpwt  air  mpd  space

Two sets of properties are displayed:

1. Dimensions of the data.
2. Formulations of the problem.

The dimensions describe the shapes of matrices as laid out in [Notation](https://pyblp.readthedocs.io/en/stable/notation.html#notation). They include:

- $T$ is the number of markets.
- $N$ is the length of the dataset (the number of products across all markets).
- $F$ is the number of firms, which we won't use in this example.
- $K_1$ is the dimension of the linear demand parameters.
- $M_D$ is the dimension of the instrument variables (excluded instruments and exogenous regressors).
- $E_D$ is the number of fixed effect dimensions (one-dimensional fixed effects, two-dimensional fixed effects, etc.).

There is only a single [`Formulation`](https://pyblp.readthedocs.io/en/stable/_api/pyblp.Formulation.html#pyblp.Formulation) for this model. 

- $X_1$ is the linear component of utility for demand and depends only on prices (after the fixed effects are removed).

### Solving the Problem

The [`Problem.solve`](https://pyblp.readthedocs.io/en/stable/_api/pyblp.Problem.solve.html#pyblp.Problem.solve) method always returns a [`ProblemResults`](https://pyblp.readthedocs.io/en/stable/_api/pyblp.ProblemResults.html#pyblp.ProblemResults) class, which can be used to compute post-estimation outputs. See the [post estimation](post_estimation.ipynb) tutorial for more information.

In [9]:
logit_results = problem.solve()
logit_results

Problem Results Summary:
GMM   Objective  Clipped  Weighting Matrix  Covariance Matrix
Step    Value    Shares   Condition Number  Condition Number 
----  ---------  -------  ----------------  -----------------
 2    +2.7E+02      0         +4.5E+07          +5.5E+04     

Cumulative Statistics:
Computation   Objective 
   Time      Evaluations
-----------  -----------
 00:00:00         2     

Beta Estimates (Robust SEs in Parentheses):
    1         prices       hpwt        air         mpd        space   
----------  ----------  ----------  ----------  ----------  ----------
 -9.9E+00    -1.5E-01    +1.3E+00    +6.8E-01    +1.8E-01    +2.4E+00 
(+2.7E-01)  (+1.2E-02)  (+4.2E-01)  (+1.4E-01)  (+4.6E-02)  (+1.3E-01)